# Placeholder base (will be extended)

In [ ]:
print('Please open the v2 file; this is a placeholder if the base is missing.')

## 18) Contribution to **Total Exports** Growth (Oil / Non‑Oil / Re‑Export)

In [ ]:

exports_plus_re = df[df["trade_type"].isin(["Export","Re-Export"])].copy()

oil_exp = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Oil (HS 27)")]
           .groupby("year", as_index=False)["value_omr"].sum()
           .rename(columns={"value_omr":"oil_export"}))

non_oil_exp = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
               .groupby("year", as_index=False)["value_omr"].sum()
               .rename(columns={"value_omr":"non_oil_export"}))

reexp = (df[df["trade_type"]=="Re-Export"]
         .groupby("year", as_index=False)["value_omr"].sum()
         .rename(columns={"value_omr":"reexport"}))

totals = (exports_plus_re.groupby("year", as_index=False)["value_omr"].sum()
          .rename(columns={"value_omr":"total_exports"}))

comp = (totals.merge(oil_exp, on="year", how="left")
               .merge(non_oil_exp, on="year", how="left")
               .merge(reexp, on="year", how="left")).fillna(0.0).sort_values("year")

comp["total_prev"] = comp["total_exports"].shift(1)
for col in ["oil_export","non_oil_export","reexport","total_exports"]:
    comp[f"delta_{col}"] = comp[col].diff()

comp["contrib_pp_oil"]      = np.where(comp["total_prev"]>0, comp["delta_oil_export"]     / comp["total_prev"] * 100, np.nan)
comp["contrib_pp_non_oil"]  = np.where(comp["total_prev"]>0, comp["delta_non_oil_export"] / comp["total_prev"] * 100, np.nan)
comp["contrib_pp_reexport"] = np.where(comp["total_prev"]>0, comp["delta_reexport"]       / comp["total_prev"] * 100, np.nan)
comp["yoy_total_%"]         = np.where(comp["total_prev"]>0, comp["delta_total_exports"]  / comp["total_prev"] * 100, np.nan)

comp[["year","total_exports","yoy_total_%","contrib_pp_oil","contrib_pp_non_oil","contrib_pp_reexport"]].head(10)


In [ ]:

import plotly.graph_objects as go
fig18 = go.Figure()
fig18.add_bar(x=comp["year"], y=comp["contrib_pp_non_oil"], name="Non‑Oil (pp)")
fig18.add_bar(x=comp["year"], y=comp["contrib_pp_oil"], name="Oil (pp)")
fig18.add_bar(x=comp["year"], y=comp["contrib_pp_reexport"], name="Re‑Export (pp)")
fig18.update_layout(barmode="relative", title="Contribution to Total Exports YoY Growth (pp) — Imports Excluded from Total",
                    xaxis_title="Year", yaxis_title="Percentage points", template="plotly_white")
fig18.show()


## 19) Contribution to **Non‑Oil Exports** Growth by BEC

In [ ]:

no_bec = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
          .groupby(["year","bec_broad"], as_index=False)["value_omr"].sum())

no_total = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
            .groupby("year", as_index=False)["value_omr"].sum()
            .rename(columns={"value_omr":"non_oil_total"}))

wide = no_bec.pivot(index="year", columns="bec_broad", values="value_omr").fillna(0.0).sort_index()
wide = wide.merge(no_total.set_index("year"), left_index=True, right_index=True, how="left")

wide["total_prev"] = wide["non_oil_total"].shift(1)
deltas = wide.drop(columns=["non_oil_total","total_prev"]).diff()
contrib_pp = deltas.div(wide["total_prev"], axis=0) * 100
contrib_pp = contrib_pp.reset_index().melt(id_vars="year", var_name="BEC", value_name="contrib_pp")

contrib_pp.head(10)


In [ ]:

import plotly.express as px
fig19 = px.bar(contrib_pp, x="year", y="contrib_pp", color="BEC",
               title="Contribution to Non‑Oil Exports YoY Growth by BEC (pp)",
               labels={"contrib_pp":"Percentage points"})
fig19.update_layout(barmode="relative", template="plotly_white")
fig19.show()


In [ ]:

last_years = contrib_pp["year"].dropna().unique()
last_years = np.sort(last_years)[-5:] if len(last_years)>5 else np.sort(last_years)

tbl = []
for y in last_years:
    sub = contrib_pp[contrib_pp["year"]==y].dropna(subset=["contrib_pp"])
    if len(sub)==0: 
        continue
    top_pos = sub.nlargest(3, "contrib_pp")
    top_neg = sub.nsmallest(3, "contrib_pp")
    for rank, row in enumerate(top_pos.itertuples(index=False), 1):
        tbl.append({"year": int(y), "type":"Top +", "rank": rank, "BEC": row.BEC, "pp": row.contrib_pp})
    for rank, row in enumerate(top_neg.itertuples(index=False), 1):
        tbl.append({"year": int(y), "type":"Top -", "rank": rank, "BEC": row.BEC, "pp": row.contrib_pp})

import pandas as pd
tbl_df = pd.DataFrame(tbl)
tbl_df


## 20) Top **Non‑Oil** Products (value & growth)

In [ ]:

latest_year = int(df["year"].dropna().max())
prev_year = latest_year - 1
print("Latest year:", latest_year, "| Previous:", prev_year)

non_oil_export = df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")].copy()

cur = (non_oil_export[non_oil_export["year"]==latest_year]
       .groupby(["hs_code","hs_name"], as_index=False)["value_omr"].sum()
       .rename(columns={"value_omr":"value_curr"}))

prev = (non_oil_export[non_oil_export["year"]==prev_year]
        .groupby(["hs_code","hs_name"], as_index=False)["value_omr"].sum()
        .rename(columns={"value_omr":"value_prev"}))

prod = cur.merge(prev, on=["hs_code","hs_name"], how="left").fillna({"value_prev":0.0})
prod["abs_delta"] = prod["value_curr"] - prod["value_prev"]
prod["yoy_pct"] = np.where(prod["value_prev"]>0, (prod["abs_delta"]/prod["value_prev"])*100, np.nan)

top_val = prod.sort_values("value_curr", ascending=False).head(10)
top_grow = prod.sort_values("abs_delta", ascending=False).head(10)

print("Top 10 by value:")
top_val[["hs_code","hs_name","value_curr","value_prev","abs_delta","yoy_pct"]]


In [ ]:

print("Top 10 by growth (absolute):")
top_grow[["hs_code","hs_name","value_curr","value_prev","abs_delta","yoy_pct"]]


In [ ]:

import plotly.express as px
fig20 = px.bar(top_val.sort_values("value_curr"), 
               x="value_curr", y="hs_name", orientation="h",
               title=f"Top 10 Non‑Oil Products by Value — {latest_year}",
               labels={"value_curr":"OMR","hs_name":""})
fig20.update_layout(template="plotly_white")
fig20.show()


## 21) HS Sections — Growth & Share over Time

In [ ]:

sec = (df[(df["trade_type"]=="Export")]
       .groupby(["year","section_name_clean","oil_flag"], as_index=False)["value_omr"].sum())

sec_no = sec[sec["oil_flag"]=="Non‑Oil"].drop(columns=["oil_flag"])
wide_sec = sec_no.pivot(index="year", columns="section_name_clean", values="value_omr").fillna(0.0).sort_index()

growth_sec = wide_sec.pct_change()*100
growth_sec = growth_sec.replace([np.inf,-np.inf], np.nan)

year_tot_no = wide_sec.sum(axis=1).replace(0, np.nan)
share_sec = wide_sec.div(year_tot_no, axis=0)*100

print("Non‑Oil section totals (wide):")
wide_sec.tail(3)


In [ ]:

print("YoY growth (%):")
growth_sec.tail(3)


In [ ]:

print("Share of non‑oil (%):")
share_sec.tail(3)


In [ ]:

import plotly.express as px
fig21 = px.imshow(growth_sec.T, aspect='auto', color_continuous_scale='RdBu', origin='lower',
                  title='YoY Growth (%) — Non‑Oil Exports by HS Section',
                  labels=dict(x="Year", y="HS Section", color="% YoY"))
fig21.update_layout(template="plotly_white")
fig21.show()


## 22) Contribution to **Non‑Oil** Growth by HS Section (pp)

In [ ]:

no_sec = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
          .groupby(["year","section_name_clean"], as_index=False)["value_omr"].sum())

no_tot = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
          .groupby("year", as_index=False)["value_omr"].sum().rename(columns={"value_omr":"non_oil_total"}))

pivot_sec = no_sec.pivot(index="year", columns="section_name_clean", values="value_omr").fillna(0.0).sort_index()
pivot_sec = pivot_sec.merge(no_tot.set_index("year"), left_index=True, right_index=True, how="left")

pivot_sec["total_prev"] = pivot_sec["non_oil_total"].shift(1)
deltas = pivot_sec.drop(columns=["non_oil_total","total_prev"]).diff()
contrib_pp_sec = deltas.div(pivot_sec["total_prev"], axis=0) * 100

contrib_long = contrib_pp_sec.reset_index().melt(id_vars="year", var_name="HS Section", value_name="pp")
contrib_long.head(10)


In [ ]:

import plotly.express as px
fig22 = px.bar(contrib_long, x="year", y="pp", color="HS Section",
               title="Contribution to Non‑Oil Exports YoY Growth by HS Section (pp)",
               labels={"pp":"Percentage points"})
fig22.update_layout(barmode="relative", template="plotly_white")
fig22.show()
